# Group Centrality Tutorial

This notebook covers the group centrality algorithms implemented in NetworKit. Group centrality measures the relative importance of a group of nodes within a graph. 

The documentation of NetworKit group centrality algorithms is available in the [centrality](https://networkit.github.io/dev-docs/python_api/centrality.html) module.

In [ ]:
import networkit as nk

In [ ]:
# Read graph
G = nk.readGraph("../input/karate.graph", nk.Format.METIS)

## Group Betweenness

Group betweenness measures the importance of a group $S$ as the fraction of shortest paths connecting pairs of non-group members that pass through $S$.

The constructor [ApproxGroupBetweenness(G, groupSize, epsilon)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=approxgr#networkit.centrality.ApproxGroupBetweenness) expects as inputs an undirected graph, the desired group size `groupSize`, and the accuracy of the approximation in `epsilon`.

In [ ]:
# Initalize algorithm
btwn = nk.centrality.ApproxGroupBetweenness(G, 10, 0.1)

In [ ]:
# Run 
btwn.run()

After running the algorithm, we can extract some information about the group betweenness centrality, e.g. the group with the highest approximate betweenness centrality score. 

In [ ]:
# Group with highest betweenness centrality
btwn.groupMaxBetweenness()

One may also have a group of nodes and is interested in the group's betweeness centrality score. The function [scoreOfGroup(group, normalized=False)]() returns the betweenness centrality score of the list of nodes in `group`.

In [ ]:
# Create an arbitrary group of nodes
group = [7, 8, 11, 20]

# Get betweenness centrality score of group
btwn.scoreOfGroup(group)

## Group Closeness

GroupCloseness implements an heuristic greedy algorithm to compute a group of nodes with high group closeness centrality. The algorithm was introduced in "Scaling up Group Closeness Maximization" Bergamini et al., ALENEX 2018.

The constructor [GroupCloseness(G, k=1, H=0)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=group#networkit.centrality.GroupCloseness) expects an unweighted graph and the desired group size in the parameter `k`. If `H` is equal to 0, simply runs the algorithm proposed in Bergamini et al.. If `H` > 0, all BFSs will explore the graph up to distance `H`. This can speed up the algorithm, at the cost of a lower solution quality.

In [ ]:
# Initalize algorithm
close = nk.centrality.GroupCloseness(G, 10)

In [ ]:
# Run
close.run()

The method [groupMaxCloseness()](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=groupma#networkit.centrality.GroupCloseness.groupMaxCloseness) returns the group of `k` nodes computed by the algorithm.

In [ ]:
# Computed group of nodes
close.groupMaxCloseness()

The [scoreOfGroup(group)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=scoreof#networkit.centrality.GroupCloseness.scoreOfGroup) method takes a group of nodes as a input, and returns the group's closeness centrality score. Hence, one can use this method to compute the group closeness centrality of a given set of nodes. The score is returned as a value between 0 and 1; a score of 1 indicates maximum group closeness centrality.

In [ ]:
# Create an arbitrary group of nodes
group = [7, 8, 11, 20]

# Get closeness centrality score of group
close.scoreOfGroup(group)

## Group Degree

Group Degree measures the importance of a group of nodes $S$ as the number of non group members that can be reached from $S$ in one hop. The algorithm implemented in NetworKit is a greedy algorithm that find an approximation of the group with the highest group degree centrality.

The constructor [GroupDegree(G, k = 1, countGroupNodes = True)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=groupdegree#networkit.centrality.GroupDegree)  expects a graph, the size of the group in the parameter `k`, and a boolean `countGroupNodes` stating if nodes inside the group should be counted in the centrality score. By default, the nodes are included in the centrality score.

In [ ]:
# Initalize algorithm
degree = nk.centrality.GroupDegree(G, 10)

In [ ]:
# Run
degree.run()

Using the [groupMaxDegree()](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=groupma#networkit.centrality.GroupDegree.groupMaxDegree) method, we can now get the group of 10 nodes computed by the algorithm.

In [ ]:
# Group with high degree centrality
degree.groupMaxDegree()

The [scoreOfGroup(group)](https://networkit.github.io/dev-docs/python_api/centrality.html?highlight=scoreof#networkit.centrality.GroupDegree.scoreOfGroup) method takes a group of nodes as a list, and returns the group's degree centrality score. Hence, one can use this method to compute the the group degree centrality of a given set of nodes.

In [ ]:
# Create an arbitrary group of nodes
group = [7, 8, 11, 20]

# Get degree centrality score of group
degree.scoreOfGroup(group)